In [20]:
import numpy as np
from scipy.optimize import minimize

def mixed_norm_polyfit(x, y, c1, c2, degree):
    """
    Поиск коэффициентов полинома, минимизирующего линейную комбинацию L1 и L2 ошибок.
    
    Потери для одной точки: c1 * |F(x_i) - y_i| + c2 * (F(x_i) - y_i)^2
    Итоговые потери: сумма по всем точкам.
    
    Аргументы:
        x : array-like, форма (n_samples,)
            Входные значения
        y : array-like, форма (n_samples,)
            Целевые значения
        c1 : float
            Коэффициент для L1-члена (должен быть >= 0)
        c2 : float
            Коэффициент для L2-члена (должен быть >= 0)
        degree : int
            Степень полинома
    
    Возвращает:
        coefficients : ndarray, форма (degree + 1,)
            Коэффициенты полинома [a0, a1, ..., a_degree] для F(x) = a0 + a1*x + ... + a_degree*x^degree
        y_pred : ndarray, форма (n_samples,)
            Предсказанные значения на входных x
    """
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    
    if x.shape[0] == 0 or y.shape[0] == 0:
        return np.zeros(degree + 1), np.array([])
    
    if x.shape[0] != y.shape[0]:
        raise ValueError("Длины x и y должны совпадать")
    
    if c1 < 0 or c2 < 0:
        raise ValueError("Коэффициенты c1 и c2 должны быть неотрицательными")
    
    if c2 == 0:
        c2 = 1e-15
    
    # Создаём матрицу Вандермонда: столбцы [x^0, x^1, ..., x^degree]
    X = np.vander(x, N=degree + 1, increasing=True)
    
    # Начальное приближение через МНК (устойчиво даже для недоопределённых систем)
    try:
        coef0, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    except np.linalg.LinAlgError:
        coef0 = np.zeros(degree + 1)
    
    # Вырожденный случай: нулевые веса
    if c1 == 0 and c2 == 0:
        return coef0, X @ coef0
    
    # Целевая функция с защитой от переполнения
    def loss(coef):
        residuals = X @ coef - y
        # Для устойчивости: избегаем возведения в квадрат очень больших чисел
        if c2 > 0 and np.any(np.abs(residuals) > 1e10):
            return np.inf
        l1_term = np.sum(np.abs(residuals))
        l2_term = np.sum(residuals ** 2) if c2 > 0 else 0.0
        return c1 * l1_term + c2 * l2_term
    
    # Основная оптимизация: метод Пауэлла (хорошо работает с негладкими функциями)
    result = minimize(
        loss,
        coef0,
        method='Powell',
        options={'maxiter': 10000, 'disp': False}
    )
    
    # Резервный метод при неудаче
    if not result.success:
        result = minimize(
            loss,
            coef0,
            method='Nelder-Mead',
            options={'maxiter': 10000, 'disp': False}
        )
    
    coefficients = result.x
    y_pred = X @ coefficients
    
    return coefficients, y_pred

In [21]:
x = np.array([0, 1, 2, 3, 4])
y = np.array([1, 3, 7, 13, 21])  # Примерно соответствует y = x² + x + 1

# Подгонка квадратичного полинома с акцентом на устойчивость к выбросам (L1)
coef, y_pred = mixed_norm_polyfit(x, y, c1=1.0, c2=0, degree=2)
print("Коэффициенты:", coef)  # Ожидаемый результат: [~1, ~1, ~1]
print("Предсказания:", y_pred)

Коэффициенты: [1. 1. 1.]
Предсказания: [ 1.  3.  7. 13. 21.]
